In [ ]:
import ee
import geemap as g
import datetime
import time
import os
import threading
#import ollama
#import notebook
import asyncio 
from IPython.core.display_functions import display
import ipywidgets as widget
auth_State = False
date_state = False
data_State = False
map_state = False
# run       python3 -m notebook      in shell to start Jupyter server
try:
    if ee.Authenticate(quiet=True):
        ee.Initialize(project="learning-project-436517")
        auth_State = True

except ValueError:
    auth_State = ValueError 
finally:
    ee.Authenticate(auth_mode="notebook")



Log = widget.Output()
Center = [48.3794, 31.1656]
global Map
#========================================================================================= Polygon Start ======================================================================================================================================

Ukraine = {
    "geodesic": False,
    "type": "Polygon",
    "coordinates": [[[
        23.32989305133208, 46.18192421358463],
        [38.66680711383208, 46.18192421358463],
        [38.66680711383208, 52.081364449517274],
        [23.32989305133208, 52.081364449517274],
        [23.32989305133208, 46.18192421358463]]]}
Mariupol = {
    "geodesic": False,
    "type": "Polygon",
    "coordinates": [[
        [37.57443050213773, 47.08504167310611],
        [37.66575435467679, 47.07709281123811],
        [37.70557979412992, 47.128505634229306],
        [37.65751460858304, 47.152792595141435],
        [37.490659750184605, 47.12943995326131],
        [37.47555354901273, 47.02750256697862],
        [37.57443050213773, 47.08504167310611]]]}
Odesa = {
    "geodesic": False,
    "type": "Polygon",
    "coordinates": [
        [
            [30.598608101814754, 46.34232305677091],
            [30.801855172127254, 46.34232305677091],
            [30.801855172127254, 46.55049603225274],
            [30.598608101814754, 46.55049603225274],
            [30.598608101814754, 46.34232305677091]
        ]
    ]
}
Gaza = {
    "geodesic": False,
    "type": "coordinates",
    "coordinates": [
        [
            [34.281418764703915, 31.161562347574517],
            [34.58766266118829, 31.541541281809828],
            [34.49153229009454, 31.608230377233596],
            [34.20863434087579, 31.327112208340928],
            [34.281418764703915, 31.161562347574517]
        ]
    ]
}
location_Index = {"Ukraine": Ukraine, "Mariupol": Mariupol, "Odesa": Odesa, "Gaza" : Gaza }
@Log.capture()
def center_coOrd(Location):  # Function to easily find center of a polygon to center map etc
    Location = location_Index[Location]
    Center = (ee.Geometry.Polygon(Location["coordinates"])).centroid().coordinates()
    print(f"Location is: {Location}   | Center is : {Center.getInfo()}")
    return Center.getInfo()


#========================================================================================= Polygon End ======================================================================================================================================



#========================================================================================= Widgets Start ======================================================================================================================================

radio_Location_Selection = widget.RadioButtons(
    options=["Ukraine", "Odesa", "Mariupol", "Gaza"],
    description= "Location Selection",
    value= "Ukraine",
    disabled=False,
    )    

display(radio_Location_Selection)


@Log.capture()
def Selection(change):  #Will check change in radio button object, "new" will == location selected which then gets put into Center_coOrd to get Lat Long and then Map is centered
    print(change)
    if change['type'] == 'change' and change['name'] == 'value':
        print(f"Changed to: {change['new']}")
        try:
            Center = center_coOrd(change["new"])
            Map.setCenter(Center[0],Center[1], 11)
            Map.remove_ee_layer
            layer_Creation()
            print("Map Updated")
        except TypeError:
            Center = [48.3794, 31.1656] # Ukraine Center as a final attemt to center map

            layer_Creation()
            print("Map Updated with default location")


        # Call your custom function here
        # your_function(change['new'])

# Attach the listener
radio_Location_Selection.observe(Selection, names='value')

#========================================================================================= Widgets End ======================================================================================================================================




#========================================================================================= Dates Start ======================================================================================================================================

py_Date = datetime.date.today().isoformat()
ee_date = ee.Date(py_Date.format())  # epoch format

Today = ee_date                     #   }-Short 14 day period for "current"  Data
Past = ee_date.advance(-14, 'day')  #   }

Past1 = ee_date.advance(-30, 'day') #   }-Short 30 day period to gather "past" Data
Past2 = ee_date.advance(-60, 'day') #   }

#========================================================================================= Dates End ======================================================================================================================================

#========================================================================================= Map Start ======================================================================================================================================


Map = g.Map(center=(Center[0],Center[1]), zoom=11) # Default location if Radio not 
#Map.add_basemap("OpenTopoMap")
display(Map)

RadarVisuals=  { "min": 0, "max": 4000 }
S2Visuals=  { "min": 0, "max": 4000 }
VisualVisuals=  { "min": 0, "max": 4000 }
BuiltVisuals=  { "min": 0, "max": 4000 }
#========================================================================================= Map End ======================================================================================================================================

#========================================================================================= Data Import Start ======================================================================================================================================
def layer_Creation():
    for x in range(2): # Loop twice for present and past dates
        Boundary = location_Index[radio_Location_Selection.value]
        if x == 0:
            Radar = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(Past, Today).filterBounds(Boundary).filter(ee.Filter.lt("resolution_meters", 13)).select("VV", "VH").filterBounds(Ukraine)
            S2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate(Past, Today).filterBounds(Boundary).filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 3)).select("SCL").filterBounds(Ukraine)
            Visual = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate(Past, Today).filterBounds(Boundary).filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5)).select("TCI_R","TCI_G","TCI_B").filterBounds(Ukraine)
            Built = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate(Past , Today).filterBounds(Boundary).select("built").mosaic().round().toInt()
            Map.add_ee_layer(Radar, RadarVisuals, "Radar Current")
            Map.add_ee_layer(S2, RadarVisuals, "S2 Current")
            Map.add_ee_layer(Visual, RadarVisuals, "Visual Current")
            Map.add_ee_layer(Built, RadarVisuals, "Built Current")
        else:
            Radar = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(Past2, Past1).filterBounds(Boundary).filter(ee.Filter.lt("resolution_meters", 13)).select("VV", "VH").filterBounds(Ukraine)
            S2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate(Past2, Past1).filterBounds(Boundary).filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 3)).select("SCL").filterBounds(Ukraine)
            Visual = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate(Past2, Past1).filterBounds(Boundary).filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5)).select("TCI_R","TCI_G","TCI_B").filterBounds(Ukraine)
            Built = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate(Past2, Past1).filterBounds(Boundary).select("built").mosaic().round().toInt()
            Map.add_ee_Layer(Radar, RadarVisuals, "Radar Past")
            Map.add_ee_layer(S2, RadarVisuals, "S2 Past")
            Map.add_ee_layer(Visual, RadarVisuals, "Visual Past")
            Map.add_ee_layer(Built, RadarVisuals, "Built Past")
            print(Map.get_layer_names)
#========================================================================================= Data Import End ======================================================================================================================================

#========================================================================================= Troubleshooting Area Start ======================================================================================================================================

print(f"Auth state: ( {auth_State} ), Data import state: ( {date_state} ), Map state: ( {map_state} ) ")
display(Log)

#========================================================================================= Troubleshooting Area End  ======================================================================================================================================



# A Landsat 8 surface reflectance image.
img = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_044034_20210508')

# Calculate normalized difference vegetation index: (NIR - Red) / (NIR + Red).
nir_band = 'SR_B5'
red_band = 'SR_B4'
ndvi = img.normalizedDifference([nir_band, red_band])

# Display NDVI result on the map.
m = geemap.Map()
m.set_center(-122.148, 37.377, 11)
m.add_layer(ndvi, {'min': 0, 'max': 0.5}, 'NDVI')
m




RadioButtons(description='Location Selection', options=('Ukraine', 'Odesa', 'Mariupol', 'Gaza'), value='Ukrain…

Map(center=[48.3794, 31.1656], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

Auth state: ( True ), Data import state: ( False ), Map state: ( False ) 


Output()